## Remodel the data to search albums by genre
We would like another view of the data where we can query by genre.  It's easy to move data from one table to another with Spark

### First create a table called albums_by_genre


In [ ]:
%%Cql create table if not exists music.albums_by_genre(album_genre text,
album_title text, performer text, album_year int, primary key(album_genre, album_title, album_year))

### Create an RDD of tuples based on the existing table

In [ ]:
case class Album (album_title: String,
                   album_genre: Option[String],
                   performer: String,
                   album_year: Int)

In [ ]:
var albums = sc.cassandraTable("music","tracks_by_album").select("album_title","album_genre","performer","album_year").as(Album)

In [ ]:
albums.first

### Some genres are null so lets map null to the string `<null>`
Let's prove there are some nulls

The ```Option``` type has some convenient methods of dealing with the None type:
- ```isEmpty``` - returns true if the value is ```None```
- ```getOrElse(<replacement value>)``` - if the value is ```None``` return the ```<replacement value>``` otherwise return the value


In [ ]:
albums.filter(a => a.album_genre.isEmpty).count

Now convert None to ```"<nothing>"```

The case class has a copy function, where we just pass functions in describing what changed
Note:  We use the copy constructor of ```a``` to create a copy of  ```a```.  When copying it, we change the value of ```album_genre```.

In [ ]:
val nonull_albums = albums.map( a => a.copy( album_genre = Some(a.album_genre.getOrElse("<nothing>"))))

### Save it to the new table
- We are using Cassandra's UPSERT behavior to de-duplicate the data

In [ ]:
nonull_albums.saveToCassandra("music","albums_by_genre",SomeColumns("album_genre","album_title","performer", "album_year"))

## View the results

In [ ]:
%%Cql select * from music.albums_by_genre limit 5